In [17]:
#load data
import pandas as pd

labels = pd.read_csv('../labels.csv', header=0, sep=';')

labels_no_useless = labels[~labels.apply(lambda x: all(int(rule) == 1 for rule in x[1:]), axis=1)]

rule_8 = labels_no_useless[labels_no_useless['Rule8'] == 1]

other_rules = labels_no_useless[labels_no_useless['Rule8'] == 0].sample(n=rule_8.shape[0]*2, random_state=42)

balanced_labels = pd.concat([rule_8, other_rules])

balanced_labels= balanced_labels[["Image_name", "Rule8"]]

balanced_labels.info()

balanced_labels.to_csv('../model_arthur/balanced_labels.csv', sep=';', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186 entries, 148 to 934
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Image_name  186 non-null    object
 1   Rule8       186 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.4+ KB


In [1]:
import os
from PIL import Image
import pandas as pd
from pandas import read_csv
import numpy as np

img_folder = "../Yoio_Park_Proof"
target_size = (512, 512)
label_data = read_csv("../model_arthur/balanced_labels.csv", sep=";")

# Function to load and resize an image
def load_image(img_path, target_size):
    with Image.open(img_path) as img:
        img = img.resize(target_size)
        img = img.convert('RGB')  # Ensure image is in RGB format
        return np.array(img)

# Read images and their corresponding labels
image_data = []
image_labels = []

for idx, row in label_data.iterrows():
    img_name = row['Image_name']
    img_path = os.path.join(img_folder, img_name)
    
    if os.path.exists(img_path):
        img_array = load_image(img_path, target_size)
        image_data.append(img_array)
        image_labels.append(row[1])  # Assuming labels are in columns after 'Image'
        print(f"Loaded image {img_name} with label {row[1]}", end='\r')
    else:
        print(f"Warning: Image {img_name} not found in folder {img_folder}")



In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image_data, image_labels, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [36]:
print(y_test.shape)

(38,)


In [37]:


import pickle

with open('../model_arthur/test_img.pickle', 'wb') as f:
    pickle.dump((X_test, y_test), f)

In [38]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)

#generate augmented images
batches = 1000
augmented_images = []
augmented_labels = []

for i in range(batches):
    batch = next(datagen.flow(X_train, y_train, batch_size=1))
    augmented_images.append(batch[0][0])
    augmented_labels.append(batch[1][0])
    print(f"Generated {i+1} images", end='\r')

: 

In [6]:
# Konvertieren der Listen in numpy Arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Kombinieren der augmentierten Bilder und Labels mit den ursprünglichen Trainingsdaten
X_train_augmented = np.concatenate((X_train, augmented_images), axis=0)
y_train_augmented = np.concatenate((y_train, augmented_labels), axis=0)

print(len(X_train_augmented))

248


In [7]:
import pickle


with open('../model_arthur/data_useable.pickle', 'wb') as f:
    pickle.dump((X_train_augmented, y_train_augmented), f)



In [16]:
import pickle
import numpy as np

# Load the augmented images and labels from the file
with open('../model_arthur/data_useable.pickle', 'rb') as f:
    augmented_images, augmented_labels = pickle.load(f)


augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

augmented_images.shape

0

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout

# Define the CNN model
# architecture: conv layer maxpooling batchnorm -> conv layer maxpooling batchnorm -> conv layer maxpooling batchnorm -> fully connected layer -> output layer
# output layer is a single neuron with sigmoid activation function
# output should be a probability between 0 and 1

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='sigmoid')
])

In [29]:
from keras.utils import to_categorical

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

history = model.fit(
  augmented_images,
  to_categorical(augmented_labels),
  validation_split=0.2,
  epochs=10,
  batch_size=32
  )



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 510, 510, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 255, 255, 32)     0         
 g2D)                                                            
                                                                 
 batch_normalization_18 (Bat  (None, 255, 255, 32)     128       
 chNormalization)                                                
                                                                 
 conv2d_19 (Conv2D)          (None, 253, 253, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 126, 126, 64)     0         
 g2D)                                                            
                                                      